# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233403229429                   -0.50    6.0
  2   -7.249490323346       -1.79       -1.39    1.0
  3   -7.251253815196       -2.75       -2.00    4.0
  4   -7.251157717262   +   -4.02       -2.02    5.0
  5   -7.251317273646       -3.80       -2.50    4.0
  6   -7.251337485165       -4.69       -3.03    4.0
  7   -7.251338792015       -5.88       -4.10    3.0
  8   -7.251338794309       -8.64       -4.24    4.0
  9   -7.251338798525       -8.38       -4.93    2.0
 10   -7.251338798702       -9.75       -5.68    3.0
 11   -7.251338798704      -11.72       -6.29    3.0
 12   -7.251338798704      -12.75       -6.52    3.0
 13   -7.251338798705      -12.91       -7.40    2.0
 14   -7.251338798705      -14.27       -7.60    4.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0810442448356497
[ Info: Arnoldi iteration step 2: normres = 0.5696356310061619
[ Info: Arnoldi iteration step 3: normres = 0.614716963627778
[ Info: Arnoldi iteration step 4: normres = 0.225945061104137
[ Info: Arnoldi iteration step 5: normres = 0.3650607677643011
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (7.60e-03, 3.68e-02, 2.98e-01, 2.05e-01, 3.07e-02)
[ Info: Arnoldi iteration step 6: normres = 0.41693522176780845
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.83e-03, 2.42e-01, 3.09e-01, 1.15e-01, 4.33e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07483251008887316
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.12e-04, 2.39e-02, 6.70e-03, 3.70e-02, 5.41e-02)
[ Info: Arnoldi iteration step 8: normres = 0.13484962434503056
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.48e-06,